In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/obesity-among-children-by-selected-characteristics/Obesity_among_children_and_adolescents_aged_2_19_years__by_selected_characteristics__United_States.csv


In [2]:
df = pd.read_csv("/kaggle/input/obesity-among-children-by-selected-characteristics/Obesity_among_children_and_adolescents_aged_2_19_years__by_selected_characteristics__United_States.csv")

In [3]:
df.head()

,INDICATOR,PANEL,PANEL_NUM,UNIT,UNIT_NUM,STUB_NAME,STUB_NAME_NUM,STUB_LABEL_NUM,STUB_LABEL,YEAR,YEAR_NUM,AGE,AGE_NUM,ESTIMATE,SE,FLAG
0,Obesity among children and adolescents aged 2-...,2-19 years,1,"Percent of population, crude",1,Total,0,0.0,2-19 years,1988-1994,1,2-19 years,0.0,10.0,0.5,NaN
1,Obesity among children and adolescents aged 2-...,2-19 years,1,"Percent of population, crude",1,Total,0,0.0,2-19 years,1999-2002,2,2-19 years,0.0,14.8,0.7,NaN
2,Obesity among children and adolescents aged 2-...,2-19 years,1,"Percent of population, crude",1,Total,0,0.0,2-19 years,2001-2004,3,2-19 years,0.0,16.3,0.8,NaN
3,Obesity among children and adolescents aged 2-...,2-19 years,1,"Percent of population, crude",1,Total,0,0.0,2-19 years,2003-2006,4,2-19 years,0.0,16.3,0.9,NaN
4,Obesity among children and adolescents aged 2-...,2-19 years,1,"Percent of population, crude",1,Total,0,0.0,2-19 years,2005-2008,5,2-19 years,0.0,16.2,0.9,NaN


In [4]:
df.drop(labels=['INDICATOR', 'UNIT', 'FLAG'], inplace=True, axis=1)

In [5]:
for col in df:    
    if df[col].dtype == "object":
        df[col] = df[col].factorize()[0]

In [6]:
df.head()

,PANEL,PANEL_NUM,UNIT_NUM,STUB_NAME,STUB_NAME_NUM,STUB_LABEL_NUM,STUB_LABEL,YEAR,YEAR_NUM,AGE,AGE_NUM,ESTIMATE,SE
0,0,1,1,0,0,0.0,0,0,1,0,0.0,10.0,0.5
1,0,1,1,0,0,0.0,0,1,2,0,0.0,14.8,0.7
2,0,1,1,0,0,0.0,0,2,3,0,0.0,16.3,0.8
3,0,1,1,0,0,0.0,0,3,4,0,0.0,16.3,0.9
4,0,1,1,0,0,0.0,0,4,5,0,0.0,16.2,0.9


In [7]:
df.shape

(840, 13)

In [8]:
df = df.dropna()

In [9]:
df.head()

,PANEL,PANEL_NUM,UNIT_NUM,STUB_NAME,STUB_NAME_NUM,STUB_LABEL_NUM,STUB_LABEL,YEAR,YEAR_NUM,AGE,AGE_NUM,ESTIMATE,SE
0,0,1,1,0,0,0.0,0,0,1,0,0.0,10.0,0.5
1,0,1,1,0,0,0.0,0,1,2,0,0.0,14.8,0.7
2,0,1,1,0,0,0.0,0,2,3,0,0.0,16.3,0.8
3,0,1,1,0,0,0.0,0,3,4,0,0.0,16.3,0.9
4,0,1,1,0,0,0.0,0,4,5,0,0.0,16.2,0.9


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.loc[:,df.columns != "SE"], df["SE"], test_size=0.3)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [11]:
# Implement Linear Regression

from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(X_train, y_train)

preds = reg.predict(X_test)

RMSE = 0

for i, entry in enumerate(preds):
    RMSE += abs(entry - y_test.iloc[i])**2
    
RMSE = (RMSE/y_test.shape[0])**(.5)

print(f"This model has an RMSE of {RMSE}\n")

This model has an RMSE of 0.43992068194660905



In [12]:
# Implement SVM

from sklearn.svm import SVR

svr_rbf = SVR(kernel="rbf", C=100, gamma=0.1, epsilon=0.1)
svr_lin = SVR(kernel="linear", C=100, gamma="auto")
svr_poly = SVR(kernel="poly", C=100, gamma="auto", degree=3, epsilon=0.1, coef0=1)

In [13]:
preds = svr_rbf.fit(X_train, y_train).predict(X_test)

RMSE = 0

for i, entry in enumerate(preds):
    RMSE += abs(entry - y_test.iloc[i])**2
    
RMSE = (RMSE/y_test.shape[0])**(.5)

print(f"This model has an RMSE of {RMSE}\n")

This model has an RMSE of 0.4164827213708397



In [14]:
preds = svr_lin.fit(X_train, y_train).predict(X_test)

RMSE = 0

for i, entry in enumerate(preds):
    RMSE += abs(entry - y_test.iloc[i])**2
    
RMSE = (RMSE/y_test.shape[0])**(.5)

print(f"This model has an RMSE of {RMSE}\n")

This model has an RMSE of 0.44687546387186133



In [15]:
preds = svr_poly.fit(X_train, y_train).predict(X_test)

RMSE = 0

for i, entry in enumerate(preds):
    RMSE += abs(entry - y_test.iloc[i])**2
    
RMSE = (RMSE/y_test.shape[0])**(.5)

print(f"This model has an RMSE of {RMSE}\n")

This model has an RMSE of 4.363716690884403

